# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

import nltk
nltk.download(['punkt','wordnet','stopwords','words'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, words

[nltk_data] Downloading package punkt to /Users/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/paul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/paul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /Users/paul/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [69]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessageToCategories',con=engine)
X = df['message'].values 
Y = df.iloc[:,4:]
#X = df.message.values
#Y = df[df.columns[4:]].values
classes = Y.columns
classes

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [ ]:
X = df['message'].values 
Y = df.iloc[:,4:]
#X = df.message.values
#Y = df[df.columns[4:]].values
classes = Y.columns
classes

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df.iloc[219,:]

id                                                                      263
message                   . .. i with limited means. Certain patients co...
original                  t avec des moyens limites. Certains patients v...
genre                                                                direct
related                                                                   2
request                                                                   0
offer                                                                     0
aid_related                                                               0
medical_help                                                              0
medical_products                                                          0
search_and_rescue                                                         0
security                                                                  0
military                                                                  0
child_alone 

In [61]:
df['related'].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [62]:
df_2 = df['related'][df['related'] == 2]
df_2

117      2
219      2
305      2
460      2
576      2
655      2
882      2
896      2
924      2
930      2
932      2
1227     2
1249     2
1310     2
1402     2
1497     2
1685     2
1779     2
2342     2
2467     2
2532     2
2548     2
3016     2
3121     2
3357     2
3611     2
4583     2
4625     2
4717     2
4776     2
        ..
12302    2
12306    2
12307    2
12308    2
12317    2
12326    2
12328    2
12335    2
12337    2
12389    2
12595    2
12668    2
13629    2
14828    2
14916    2
15493    2
15814    2
15860    2
15927    2
16798    2
17401    2
18251    2
18514    2
19714    2
20051    2
20324    2
20495    2
22326    2
23382    2
25215    2
Name: related, Length: 188, dtype: int64

In [5]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'],
      dtype=object)

In [6]:
Y[:10]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
len(X)

26216

In [8]:
len(Y)

26216

### 2. Write a tokenization function to process your text data

In [129]:
def tokenize(text):
    #wrds = set(words.words())
    text = re.sub(r"[^a-zA-Z]"," ",text).lower()
    tokens = word_tokenize(text)
    #tokens = [t for t in tokens if t not in stopwords.words('english')]
    #tokens = [t for t in tokens if t in wrds]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [130]:
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

Information about the National Palace-
['information', 'about', 'the', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank you!
['plea

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [134]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

In [139]:
#pipeline = Pipeline([
#                ('vect', CountVectorizer(tokenizer=tokenize)),
#                ('tfidf', TfidfTransformer()),
#                ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
#                ])

pipeline = Pipeline([
                #('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', HashingVectorizer(tokenizer=tokenize)),
                ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(),n_jobs=1)))
                ])

#pipeline = Pipeline([
#               ('vect', CountVectorizer(ngram_range=(1,3), max_df=0.50 ) ),
#               ('tfidf', TfidfTransformer() ),
#               ('clf', OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
#                                          alpha=1e-4, n_iter=5, random_state=42,
#                                        shuffle=True, n_jobs=-1) )),
#               ])

In [13]:
#vect = CountVectorizer(tokenizer=tokenize)
#X_train = vect.fit_transform(X_train)

In [12]:
#dict = vect.vocabulary_
#sorted(dict.items(), key=lambda kv: kv[1], reverse=True)

In [42]:
'''sorted(dict.items())
new_list = {}
words = set(nltk.corpus.words.words())
for (key,value) in dict.items():
    if key in words:
        new_list.update([(key,value)])
len(new_list)'''

11295

In [13]:
#dict

In [14]:
#sorted(dict.items(), key=lambda kv: kv[1])

In [60]:
#tfidf = TfidfTransformer()
#X_train_tfidf = tfidf.fit_transform(X_train)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [140]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.33,random_state=19)
pipeline.fit(X_train,Y_train)

/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('tfidf', HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_range=(1, 1), non_negative=False,
         norm='l2', p...'l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=None))])

In [28]:
#clf = RandomForestClassifier()
#model = MultiOutputClassifier(clf)
#model.fit(X_train_tfidf,Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [141]:
Y_pred = pipeline.predict(X_test)

In [82]:
Y_pred.shape

(5244, 36)

In [89]:
#X_test_counts = vect.transform(X_test)
#X_test_tfidf = tfidf.transform(X_test_counts)
#Y_pred = model.predict(X_test_tfidf)

In [142]:
overall_accuracy = (Y_pred == Y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 95.09% 



In [75]:
#print(classification_report(Y_test[1],Y_pred[1],labels=36, target_names=classes))
#print(classification_report(Y_test.iloc[:,0:].values, np.array([x[0:] for x in Y_pred]), target_names=classes))
#print(classification_report(Y_test, Y_pred))

In [143]:
Y_pred_df = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_df[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.67      0.54      0.60      2033
           1       0.86      0.92      0.89      6557
           2       0.67      0.16      0.26        62

   micro avg       0.82      0.82      0.82      8652
   macro avg       0.73      0.54      0.58      8652
weighted avg       0.82      0.82      0.82      8652

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      7161
           1       0.79      0.60      0.68      1491

   micro avg       0.90      0.90      0.90      8652
   macro avg       0.85      0.78      0.81      8652
weighted avg       0.90      0.90      0.90      8652

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8505
           1       0.74      0.33      0.45       147

   micro avg       0.99      0.99      0.99      8652
   macro avg       0.86      0.66      0.72      8652
weighted avg       0.98      0.99      0.98      8652

------------------------------------------------------

FEATURE: money

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8455
           1       0.54      0.16      0.24       197

   micro avg       0.98      0.98      0.98      8652
   macro avg       0.76      0.58      0.62      8652
weighted avg       0.97      0.98      0.97      8652

------------------------------------------------------

FEATURE: missing_people

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8543
           1       0.78      0.13      0.22       109

   micro avg       0.99      

In [72]:
Y_test.iloc[:,0:].values.shape

(8652, 36)

In [73]:
np.array([x[0:] for x in Y_pred]).shape

(8652, 36)

In [79]:
print("\t\t Accuracy Score: ")
print("\t\t ---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))
print("\t\t ---------------")    
print('\t Overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

		 Accuracy Score: 
		 ---------------
                  related : 0.81
                  request : 0.89
                    offer : 1.00
              aid_related : 0.74
             medical_help : 0.92
         medical_products : 0.96
        search_and_rescue : 0.97
                 security : 0.98
                 military : 0.97
              child_alone : 1.00
                    water : 0.96
                     food : 0.93
                  shelter : 0.93
                 clothing : 0.98
                    money : 0.98
           missing_people : 0.99
                 refugees : 0.97
                    death : 0.96
                other_aid : 0.87
   infrastructure_related : 0.93
                transport : 0.96
                buildings : 0.95
              electricity : 0.98
                    tools : 0.99
                hospitals : 0.99
                    shops : 1.00
              aid_centers : 0.99
     other_infrastructure : 0.96
          weather_related : 0.86
    

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
parameters = {'clf__estimator__n_estimators':[10,20],
              'clf__estimator__min_samples_split':[2,4]
              }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv

param_grid = dict(reduce_dim=[None, PCA(5), PCA(10)],
...                   clf=[SVC(), LogisticRegression()],
...                   clf__C=[0.1, 10, 100])
>>> grid_search = GridSearchCV(pipe, param_grid=param_grid)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
Y_pred = cv.predict(X_test)

In [28]:
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in Y_pred]), target_names=classes))
print("         Accuracy Score: ")
print("         ---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))

                        precision    recall  f1-score   support

               related       0.85      0.43      0.57      1105
               request       0.00      0.00      0.00        25
                 offer       0.76      0.57      0.65      2653
           aid_related       0.64      0.09      0.15       494
          medical_help       0.74      0.05      0.10       325
      medical_products       0.75      0.05      0.09       196
     search_and_rescue       0.00      0.00      0.00       122
              security       0.67      0.07      0.12       213
              military       0.00      0.00      0.00         0
           child_alone       0.89      0.37      0.52       426
                 water       0.86      0.43      0.57       679
                  food       0.83      0.25      0.38       571
               shelter       0.75      0.10      0.18        86
              clothing       1.00      0.03      0.06       162
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.